In [3]:
%%time
import astropy.io.fits as fits
from astropy.convolution import Gaussian2DKernel
import matplotlib ; import matplotlib.pyplot as pyplot
import numpy as np
import statmorph
import photutils
from photutils.segmentation import SourceCatalog
import scipy.ndimage as ndi
import warnings; warnings.simplefilter('ignore')
import time 
from astropy.convolution import convolve

CPU times: user 27 µs, sys: 1 µs, total: 28 µs
Wall time: 28.8 µs


In [2]:
%%time
def compute_morphs(image):
    
    threshold = photutils.detect_threshold(image, 2.5)
    npixels = 10  # minimum number of connected pixels
    kernel_sigma = 3.0 / (2.0 * np.sqrt(2.0 * np.log(2.0)))  # FWHM = 3
    kernel = Gaussian2DKernel(kernel_sigma, x_size=3, y_size=3)
    kernel.normalize()
    # Detect 
    segm = photutils.detect_sources(image, threshold, npixels, filter_kernel=kernel)
    
    cat = SourceCatalog(image, segm)
    tbl = cat.to_table()
    
    totals=tbl['segment_flux']

    labels = tbl['label']
    segm.keep_labels(labels[totals > 1e6])
    
    source_morphs = statmorph.source_morphology(image, segm, gain=5)
    
    d = []
    m = []
    s = []
    c = []
    g = []
    a = [] 
    b = []
    
    flux = []
    for sm in source_morphs:
        d.append(sm.deviation)
        m.append(sm.m20)
        s.append(sm.gini_m20_merger)
        c.append(sm.concentration)
        g.append(sm.gini) 
        a.append(sm.asymmetry)
        b.append(sm.gini_m20_bulge)
        flux.append(sm.flux_circ)
        
    print(d)
    
    d = np.asarray(d) 
    m = np.asarray(m) 
    s = np.asarray(s) 
    c = np.asarray(c) 
    g = np.asarray(g) 
    a = np.asarray(a) 
    b = np.asarray(b)
    flux = np.asarray(flux)
    
    
    return d, m, s, flux, c, g, a, b 

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
